In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.regularizers import *

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import wandb

In [ ]:
bs = 32
img_size = (224, 224)

tn_gen = ImageDataGenerator(
    horizontal_flip=True
    , brightness_range=[0.8, 1.1]
    , rotation_range=10
    , featurewise_center=True
    , featurewise_std_normalization=True
    , fill_mode='constant'
)
val_gen = ImageDataGenerator(
    # None
)

tn_set = tn_gen.flow_from_directory(
    directory='./CUB_200_2011/train'
    , shuffle=True
    , target_size=img_size
    , class_mode='categorical'
    , batch_size=bs
)

val_set = val_gen.flow_from_directory(
    directory='./CUB_200_2011/valid'
    , target_size=img_size
    , class_mode='categorical'
    , batch_size=bs
)

# fit_generator 사용

In [13]:
import tensorflow as tf

class Mish(tf.keras.layers.Layer):
    def __init__(self):
        super(Mish, self).__init__()
        
    def call(self, inputs):
        return inputs * tf.math.tanh(tf.math.softplus(inputs))

In [14]:
class Dense_Layer(tf.keras.layers.Layer):
    def __init__(self
                , filters
                , kernel_size=3
                , padding='same'
                , strides=1
                , activation=Mish
                , **kwargs):
        super(Dense_Layer, self).__init__()
        
        self.activation=activation()
        self.strides=strides
        
        # Dense Block
        self.BN1 = BatchNormalization()
        self.Conv1 = Conv2D(filters * 4, 1, padding=padding, strides=1)
        
        self.BN2 = BatchNormalization()
        self.Conv2 = Conv2D(filters, 3, padding=padding, strides=1)
        
        
    def call(self, inputs):
        out = inputs
        
        out = self.activation(out)
        out = self.BN1(out)
        out = self.Conv1(out)
        
        out = self.activation(out)
        out = self.BN2(out)
        out = self.Conv2(out)
        
        
        out = Concatenate(axis=-1)([inputs, out])
        
        return out

In [15]:
class Transition_Layer(tf.keras.layers.Layer):
    def __init__(self
                , filters):
        super(Transition_Layer, self).__init__()
        
        self.BN = BatchNormalization()
        #self.act = ReLU()
        self.act = Mish()
        self.conv = Conv2D(filters, 1)
        self.avg_pool = AveragePooling2D(pool_size=2, strides=2)
        
    def call(self, inputs):
        out = inputs
        
        out = self.BN(out)
        out = self.act(out)
        out = self.conv(out)
        out = self.avg_pool(out)
        
        return out

In [16]:
def DenseNet_121(n_classes=200):
    # STEM layer
    inputs = Input(shape=(224, 224, 3))
    out = inputs
    out = Conv2D(64, 7, padding='same', strides=2)(out)
    out = MaxPool2D((3,3), strides=2, padding='same')(out)
        
    prev = 64
    for filters in [64, 128, 256, 512]: # dense layers
        for blocks in [6, 12, 24, 16]:
            out = Dense_Layer(filters)(out)
        out = Transition_Layer(filters)(out)
        
    out = GlobalAveragePooling2D()(out)
    out = Dense(1024, activation='relu')(out)
    out = Dense(512, activation='relu')(out)
    out = Dense(200, activation='softmax', kernel_regularizer=L2())(out)
    
    return Model(inputs, out)

model = DenseNet_121()
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_37 (Conv2D)          (None, 112, 112, 64)      9472      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 dense__layer_16 (Dense_Laye  (None, 56, 56, 128)      165440    
 r)                                                              
                                                                 
 dense__layer_17 (Dense_Laye  (None, 56, 56, 192)      182080    
 r)                                                              
                                                           

In [17]:
wandb.init(
    # set the wandb project where this run will be logged
    project="bird_returns2",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.001,
    "architecture": "CNN"
    }
)

accuracy,▁▁▂▂▃▃▄▄▅▅▆▆▆▇▇▇▇███████████████████████
loss,█▇▆▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▂▂▃▃▄▅▆▅▆▆▆▆▆▇▆▇▇▆▇▇▇▇▇█▇█▇▇███▇▇█████
val_loss,██▆▆▄▄▃▂▂▃▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▁▂▂▂▂▁▁▁▂▂▁▁▁▁▁
accuracy,0.99857
loss,0.05445
val_accuracy,0.5626
val_loss,2.21659


In [18]:
import wandb
from tensorflow.keras.callbacks import Callback

class WandbCallback(Callback):
    def __init__(self):
        super().__init__()

    def on_epoch_end(self, epoch, logs=None):
        wandb.log(logs)

wandbcallback = WandbCallback()

In [19]:
LR = 0.01

model.compile(loss='categorical_crossentropy'
             , optimizer=tf.keras.optimizers.SGD(learning_rate=LR)
             , metrics=['accuracy'])

In [ ]:
model.fit_generator(tn_set
                    , validation_data=val_set
                    , epochs=100
                    , use_multiprocessing=True
                    , workers=15
                    , callbacks=[wandbcallback])

/tmp/ipykernel_240710/3167582973.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(tn_set


Epoch 1/100
164/164 [==============================] - 79s 432ms/step - loss: 8.0214 - accuracy: 0.0131 - val_loss: 7.9931 - val_accuracy: 0.0061
Epoch 2/100
164/164 [==============================] - 70s 405ms/step - loss: 7.6171 - accuracy: 0.0293 - val_loss: 7.7504 - val_accuracy: 0.0122
Epoch 3/100
164/164 [==============================] - 72s 421ms/step - loss: 7.2086 - accuracy: 0.0446 - val_loss: 8.3230 - val_accuracy: 0.0122
Epoch 4/100
164/164 [==============================] - 72s 417ms/step - loss: 6.8233 - accuracy: 0.0627 - val_loss: 6.9428 - val_accuracy: 0.0489
Epoch 5/100
164/164 [==============================] - 73s 420ms/step - loss: 6.4504 - accuracy: 0.0844 - val_loss: 6.8036 - val_accuracy: 0.0382
Epoch 6/100
164/164 [==============================] - 72s 411ms/step - loss: 6.1130 - accuracy: 0.0985 - val_loss: 7.0591 - val_accuracy: 0.0458
Epoch 7/100
164/164 [==============================] - 71s 403ms/step - loss: 5.7836 - accuracy: 0.1266 - val_loss: 6.3930 -